In [25]:
import pandas as pd
import re

In [26]:
mitula = pd.read_csv("/home/mario/Proyectos_Mario/proyecto_alonso/datos_extraidos/casas_mitula.csv")

In [27]:
# Se eliminan todas las casas que tienen un remate bancario o hipotecario
# ya que no representan el valor real 
casas_remate = list(mitula[mitula.description.str.contains("remate|bancaria|recuperación|hipotecaria", case=False)].index)
mitula = mitula.drop(index=casas_remate)
mitula = mitula.reset_index(drop="first")

In [28]:
# Hay columnas con datos nulos
mitula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             108 non-null    object 
 1   seller            108 non-null    object 
 2   property_type     108 non-null    object 
 3   address           108 non-null    object 
 4   price             108 non-null    int64  
 5   bedrooms          93 non-null     float64
 6   bathrooms         93 non-null     float64
 7   built_area        89 non-null     float64
 8   land_area         91 non-null     float64
 9   description       108 non-null    object 
 10  publication_date  108 non-null    object 
dtypes: float64(4), int64(1), object(6)
memory usage: 9.4+ KB


In [29]:
# Al parecer todos los datos pertenecen a casas
mitula.property_type.unique()

array(['Casa', 'Terreno', 'Locales Comerciales', 'Villa',
       'Casas En Condominios', 'Edificio'], dtype=object)

In [30]:
# Se eliminan los datos nulos
mitula = mitula.dropna(ignore_index=True)

In [31]:
# Adecuo los valores de la columna adddress para hacer una búsqueda efectiva de los zip codes
mitula["address"] = mitula["address"].apply(lambda x: x.lower()\
                                                        .replace("á", "a")\
                                                        .replace("é", "e")\
                                                        .replace("í", "i")\
                                                        .replace("ó", "o")\
                                                        .replace("ú", "u"))
mitula["address"] = mitula["address"].str.replace("1ra secc", "1a secc")

### Creando la columna zip_code

In [32]:
# Leo la tabla de códigos postales
tabla_CP = pd.read_html("https://codigo-postal.co/mexico/oaxaca/oaxaca-de-juarez/")
tabla_CP = tabla_CP[0]

# Seleccionar las columas que me sirven 
tabla_CP = tabla_CP.iloc[:, :2]
tabla_CP = tabla_CP.drop_duplicates(ignore_index=True)

# Guardar en listas las colonias y sus C.P.
cp = list(tabla_CP.iloc[:, 0])
direccion = list(tabla_CP.iloc[:, 1])

# Crear un diccionario y Eliminar acentos de los nombres de las direcciones
cp_dict = {}
for i, j in zip(direccion, cp):
    cp_dict[i.lower()\
        .replace("á", "a")\
        .replace("é", "e")\
        .replace("í", "i")\
        .replace("ó", "o")\
        .replace("ú", "u")] = j

In [33]:
# Creo una función que asigna el zip code según la dirección
def buscador_zip_code(lista, diccionario):
    lista_coincidencias = []
    for i in lista:
        if (i.strip() == 'oaxaca'):
            pass
        else:
            regex = re.compile(fr"(.{{0,30}}{i.strip()}.{{0,30}})", re.IGNORECASE)
            coincidencia = list(filter(lambda x: regex.findall(x), diccionario.keys()))
            if len(coincidencia) > 0:
                lista_coincidencias.append(coincidencia[0])
    try:
        return diccionario["".join(lista_coincidencias[0])]
    except:
        None

In [34]:
mitula["zip_code"] = mitula["address"].apply(lambda x: x.split(","))
mitula["zip_code"] = mitula["zip_code"].apply(buscador_zip_code, diccionario=cp_dict)

In [35]:
# Existen dos filas con valores nulos en la columna zip_code
mitula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             74 non-null     object 
 1   seller            74 non-null     object 
 2   property_type     74 non-null     object 
 3   address           74 non-null     object 
 4   price             74 non-null     int64  
 5   bedrooms          74 non-null     float64
 6   bathrooms         74 non-null     float64
 7   built_area        74 non-null     float64
 8   land_area         74 non-null     float64
 9   description       74 non-null     object 
 10  publication_date  74 non-null     object 
 11  zip_code          72 non-null     float64
dtypes: float64(5), int64(1), object(6)
memory usage: 7.1+ KB


In [36]:
# Se eliminan los valores nulos
mitula = mitula.dropna(ignore_index=True)

### Creando la columna parking

In [37]:
# Con esta función localizo el texto donde especifica los lugares de estacionamiento en la descripción
def econtrar_textos(texto):
    regex = r"(?i)(.{0,10}estacionamientos?.{0,30}|.{0,10}cocheras?.{0,30}|.{0,10}garages?.{0,30}|.{0,10}parkings?.{0,30})"
    expresion = re.findall(regex, texto)
    expresion = "".join(expresion)
    if len(expresion) > 0:
        valor_de_celda = expresion
    else:
        valor_de_celda = "cero"
    return valor_de_celda

In [38]:
# Función para encontrar en la descripción el número de lugares de estacionamiento
def asignar_valor_estacionamiento(texto):
    # Buscar dígitos numéricos en el texto
    if texto != "cero":    
        numeros_encontrados = re.findall(r'\d+', texto)

        if numeros_encontrados:
            # Tomar el primer número encontrado y convertirlo a entero
            return int(numeros_encontrados[0])
        elif "para dos" in texto.lower():
            return 2
        elif "para tres" in texto.lower():
            return 3
        elif "para cuatro" in texto.lower():
            return 4
        elif "para cinco" in texto.lower():
            return 5
        else:
            return 1

In [39]:
# Aplicar las funciones para buscar los valores de parking
mitula["parking"] = mitula["description"].apply(econtrar_textos)
mitula["parking"] = mitula["parking"].apply(asignar_valor_estacionamiento)

In [40]:
# Se tienen algunos valores nulos en la columna parking
mitula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             72 non-null     object 
 1   seller            72 non-null     object 
 2   property_type     72 non-null     object 
 3   address           72 non-null     object 
 4   price             72 non-null     int64  
 5   bedrooms          72 non-null     float64
 6   bathrooms         72 non-null     float64
 7   built_area        72 non-null     float64
 8   land_area         72 non-null     float64
 9   description       72 non-null     object 
 10  publication_date  72 non-null     object 
 11  zip_code          72 non-null     float64
 12  parking           49 non-null     float64
dtypes: float64(6), int64(1), object(6)
memory usage: 7.4+ KB


In [41]:
# Se eliminan los valores nulos
mitula = mitula.dropna(ignore_index=True)

In [42]:
# El valor de la propiedad con el terreno más grande no es una casa
# es un conjunto de departamentos
# El resto de proiedades tienen una extensión muy alejada de ésta
mitula.sort_values(by="land_area", ascending=False).head(5)

,title,seller,property_type,address,price,bedrooms,bathrooms,built_area,land_area,description,publication_date,zip_code,parking
35,PROPIEDAD CON 6 DEPARTAMENTOS Y CASA - (3),Mitula,Casa,"oaxaca, oaxaca, reforma",30000000,10.0,10.0,760.0,839.0,PROPIEDAD MUY BIEN UBICADA CON 6 DEPARTAMENTOS...,"Hace 2 semanas, 3 días - Publicado por REMAX c...",68050.0,1.0
36,Casa en Venta en la Colonia Volcanes - (3),Mitula,Casa,"oaxaca, oaxaca, volcanes",7400000,6.0,4.0,350.0,366.0,"Propiedad en la colonia Volcanes, Oaxaca, cuen...",1 jun 2023 - Publicado por REMAX cantera,68020.0,5.0
30,"VENTA DE CASA EN FRAMBOYANES, OAXACA.",Mitula,Casa,"trinidad de las huertas, oaxaca de juarez, oax...",630000,3.0,2.0,400.0,335.0,MAGNIFICA OPORTUNIDAD DE ADQUIRIR BONITA CASA ...,"Hace 1 semana, 2 días - Publicado por Peza Con...",68080.0,1.0
47,AMPLIA CASA EN ZONA NORTE VENTA A 10 MIN DEL C...,Mitula,Casa,"volcanes, oaxaca de juarez, oaxaca, mexico",4700000,3.0,1.0,203.0,322.0,AMPLIA CASA EN ZONA NORTE EN VENTAMuy cerca de...,26 abr 2023 - Publicado por AGENTES DE BIENES ...,68020.0,4.0
45,"VENTA DE CASA EN GUADALUPE VICTORIA, OAXACA, O...",Mitula,Casa,"macedonio alcala, guadalupe victoria, oaxaca d...",560000,3.0,2.0,250.0,300.0,Hermosa propiedad en venta.La propiedad consta...,"Hace 2 semanas, 4 días - Publicado por GP SERV...",68033.0,1.0


In [43]:
# Se elimina la propiedad con el terreno más grande ya que representa 
# un conjunto de departamentos
mitula = mitula.drop(index=mitula.land_area.argmax())
mitula = mitula.drop_duplicates()
mitula = mitula.reset_index(drop="first")

In [44]:
mitula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             43 non-null     object 
 1   seller            43 non-null     object 
 2   property_type     43 non-null     object 
 3   address           43 non-null     object 
 4   price             43 non-null     int64  
 5   bedrooms          43 non-null     float64
 6   bathrooms         43 non-null     float64
 7   built_area        43 non-null     float64
 8   land_area         43 non-null     float64
 9   description       43 non-null     object 
 10  publication_date  43 non-null     object 
 11  zip_code          43 non-null     float64
 12  parking           43 non-null     float64
dtypes: float64(6), int64(1), object(6)
memory usage: 4.5+ KB


In [45]:
mitula.to_csv("datos_mitula.csv", index=False)